In [9]:
# %load generate_subset_B.py
import pandas as pd
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn import cross_validation, preprocessing, metrics


# Read in combined dfs
df_train = pd.read_csv("subset_A_train.csv")
df_test = pd.read_csv("subset_A_test.csv")

df_train_support = pd.read_csv('train.csv')
df_test_support = pd.read_csv('test.csv')

#get NumMosquitos from original training file
df_train['NumMosquitos'] = df_train_support.NumMosquitos

# Assign unspecified culex to pipiens / restuans
df_test.Species[df_test.Species == 'UNSPECIFIED CULEX'] = 'CULEX PIPIENS/RESTUANS'

# Create copy of train df and drop string features
string_features = ['Address','Street','AddressNumberAndStreet','CodeSum']

X = df_train.copy()
X = X.drop(string_features,axis=1)

# Create groups for multiple rows which have the same value

groups = ['Date','Trap','Species']
tmp = X.groupby(groups)['NumMosquitos'].sum()
X_tmp = X.join(X.groupby(groups)['NumMosquitos'].sum(), on= groups,rsuffix='_counter')

# New WnvPresent
y = X_tmp.NumMosquitos_counter

#X_tmp = X_tmp.drop(['Trap',
#                    'Date',
#                    'NumMosquitos',
#                    'NumMosquitos_counter',
#                    'WnvPresent',
#                    'counter',
#                    'Unnamed: 0'
#])


# Species to labels
# TODO dummy variables might be better!!

#X_tmp.Species = species_encoder.fit_transform(X.Species)
#df_test.Species = species_encoder.transform(df_test.Species)
#df_train.species = species_encoder.transform(df_train.Species)


#create dummies

s = pd.Series(list(test.Species))
dummies = pd.get_dummies(s)
test.drop(['Species', 'Depth'], axis=1, inplace=True)
test = pd.concat([dummies, test], axis=1)





df_train['count_mosquitos'] = y
df_train = df_train.drop(['NumMosquitos'],axis=1)

# Add conditional features
for feature in df_train.columns[:-1]:
    featureprior = df_train.groupby(
        [feature])['WnvPresent'].sum() /df_train.groupby([feature])['WnvPresent'].count()
    df_train = df_train.join(featureprior,
                             on=feature,
                             rsuffix='_conditional_' + feature)
    df_test = df_test.join(featureprior,
                           on=feature,
                           rsuffix = '_conditional_' + feature)


df_test = df_test.dropna(axis=1)#.shape    

not_droplist = [var for var in df_test.columns]
not_droplist.append('WnvPresent')

droplist = [var for var in df_train.columns if var not in not_droplist]
df_train = df_train.drop(droplist,axis=1)

df_train.to_csv('subset_B_train.csv')
df_test.to_csv('subset_B_test.csv')


/home/thomas/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
[var for var in df_train.columns if var not in df_test.columns]

['WnvPresent']

In [17]:
df_train.count_mosquitos

AttributeError: 'DataFrame' object has no attribute 'count_mosquitos'